In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Differential directions for mESC_Pj data

In [ ]:
from IPython import display
import pandas as pd
import numpy as np
import datetime
from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
from tensorflow import keras
import tensorflow as tf
from keras.callbacks import EarlyStopping
import tensorflow_probability as tfp
import time
import pickle
import os

#Load Data
Root_Folder = "/content/drive/MyDrive"
data = 'gastr'
tech = '10X'
data_parent_folder = "/Data"
code_parent_folder = "/scMVI"

os.chdir('/content/drive/My Drive/' + code_parent_folder +'/Code')

Rna_train = pd.read_pickle(Root_Folder + data_parent_folder + "/Data_ATAC/Rna_train_"+data+".pickle")
Rna_nrm_train = pd.read_pickle(Root_Folder + data_parent_folder + "/Data_ATAC/Nrm_rna_train_"+data+".pickle")

Rna_test = pd.read_pickle(Root_Folder + data_parent_folder + "/Data_ATAC/Rna_test_"+data+".pickle")
Rna_nrm_test = pd.read_pickle(Root_Folder + data_parent_folder + "/Data_ATAC/Nrm_rna_test_"+data+".pickle")

Rna = pd.concat([Rna_train,Rna_test],axis=0)

In [ ]:
Rna_tr = tf.convert_to_tensor(Rna_train,dtype=tf.float32)
Rna_tst = tf.convert_to_tensor(Rna_test,dtype=tf.float32)

Rna_nrm_tr = tf.math.log(tf.convert_to_tensor(Rna_nrm_train,dtype=tf.float32))
Rna_nrm_tst = tf.math.log(tf.convert_to_tensor(Rna_nrm_test,dtype=tf.float32))

Rna_nrm_tr = tf.repeat(Rna_nrm_tr,repeats=Rna_tr.shape[1],axis=1)
Rna_nrm_tst = tf.repeat(Rna_nrm_tst,repeats=Rna_tst.shape[1],axis=1)

training_size=Rna_tr.shape[0]
testing_size=Rna_tst.shape[0]
batch_size =  10

inp_rna_sz = Rna_tr.shape[1]

print(training_size)
print(inp_rna_sz)

36726
6079


In [ ]:
from Models import scMVI_tp2
from Obj_Blocks import *

In [ ]:
#Define model and load weights for the trained model
mdl = scMVI_tp2(inp_tp2_dim=inp_rna_sz,intermidiate_dim=100,latent_dimensions=10)
mdl.load_weights(Root_Folder + code_parent_folder + "/Networks/scMVI_10X_"+data+"_rna/scMVI_10X_"+data+"_rna")

In [ ]:
#Functions for estimating differential directions and stats for experiments outlined in the main chapter 
def index_creation(meta_data,group_by,group_1,smps_group_1,group_2,smps_group_2):


                        cell_num = meta_data.shape[0]
                        group_index_1 = np.zeros((cell_num,))
                        group_index_2 = np.zeros((cell_num,))

                        ind_1 = np.where(meta_data[group_by]==group_1)[0]
                        ind_2 = np.where(meta_data[group_by]==group_2)[0]

                        if np.logical_or(len(ind_1)<smps_group_1,len(ind_2)<smps_group_2):

                                    raise ValueError('samples for groups 1 and 2 cannot be more than cells in groups 1 and 2')        



                        perm_ind_1 = ind_1[np.random.permutation(len(ind_1))[:smps_group_1]]
                        perm_ind_2 = ind_2[np.random.permutation(len(ind_2))[:smps_group_2]]

                        group_index_1[perm_ind_1]=1
                        group_index_2[perm_ind_2]=1

                        group_index_1 = tf.squeeze(tf.where(group_index_1==1))
                        group_index_2 = tf.squeeze(tf.where(group_index_2==1))

                        return group_index_1, group_index_2

def create_exp(metadata,grp_1,smps_1,grp_2,smps_2):

        smps_group_1 =[smps_1,int(np.round(smps_1*0.2)),smps_1,int(np.round(smps_1*0.2)),int(np.round(smps_1*0.5)),int(np.round(smps_1*0.5)),int(np.round(smps_1*0.2))]
        smps_group_2 =[smps_2,smps_2,int(np.round(smps_2*0.2)),int(np.round(smps_2*0.5)),int(np.round(smps_2*0.2)),int(np.round(smps_2*0.5)),int(np.round(smps_2*0.2))]

        col_all_rna = []

        for jj in range(5):


              group_index_11,group_index_21 = index_creation(metadata,'stage',grp_1,smps_group_1[0],grp_2,smps_group_2[0])
              group_index_12,group_index_22 = index_creation(metadata,'stage',grp_1,smps_group_1[1],grp_2,smps_group_2[1])
              group_index_13,group_index_23 = index_creation(metadata,'stage',grp_1,smps_group_1[2],grp_2,smps_group_2[2])
              group_index_14,group_index_24 = index_creation(metadata,'stage',grp_1,smps_group_1[3],grp_2,smps_group_2[3])
              group_index_15,group_index_25 = index_creation(metadata,'stage',grp_1,smps_group_1[4],grp_2,smps_group_2[4])
              group_index_16,group_index_26 = index_creation(metadata,'stage',grp_1,smps_group_1[5],grp_2,smps_group_2[5])
              group_index_17,group_index_27 = index_creation(metadata,'stage',grp_1,smps_group_1[6],grp_2,smps_group_2[6])

              a1_rna = mdl.Diff_Dir_Dist_Avg(xxxx,group_index_11,group_index_21,'all')
              a2_rna = mdl.Diff_Dir_Dist_Avg(xxxx,group_index_12,group_index_22,'all')
              a3_rna = mdl.Diff_Dir_Dist_Avg(xxxx,group_index_13,group_index_23,'all')
              a4_rna = mdl.Diff_Dir_Dist_Avg(xxxx,group_index_14,group_index_24,'all')
              a5_rna = mdl.Diff_Dir_Dist_Avg(xxxx,group_index_15,group_index_25,'all')
              a6_rna = mdl.Diff_Dir_Dist_Avg(xxxx,group_index_16,group_index_26,'all')
              a7_rna = mdl.Diff_Dir_Dist_Avg(xxxx,group_index_17,group_index_27,'all')

              col_all_rna.append([a1_rna,a2_rna,a3_rna,a4_rna,a5_rna,a6_rna,a7_rna])

              print(100*(1+jj)/5.0)


        return col_all_rna

def collect_stats(x):

        x_med = tfp.stats.percentile(tf.squeeze(x),50,axis=2)
        x_med_mn = tf.reduce_mean(tf.reduce_mean(x_med,axis=1),axis=0)[tf.newaxis,:]
        x_med_std = tf.math.sqrt(tf.reduce_mean(tfp.stats.variance(x_med,sample_axis=1),axis=0))[tf.newaxis,:]
        x_std = tf.math.sqrt(tf.reduce_mean(tfp.stats.variance(tf.squeeze(x),sample_axis=2),axis=0))

        x_med_rng = tf.reduce_mean(tf.reduce_max(x_med,axis=1) -tf.reduce_min(x_med,axis=1),axis=0)[tf.newaxis,:]

        return x_med_mn,x_med_std,x_std,x_med_rng


In [ ]:
Rna_comb = tf.concat([Rna_tr,Rna_tst],axis=0)
Rna_nrm_comb = tf.concat([Rna_nrm_tr,Rna_nrm_tst],axis=0)
xxxx = [Rna_comb,Rna_nrm_comb]

In [ ]:
#Load Metadata
if data == "gastr":
      if tech == "10X":
                  gastr_ind_drp = [0,1]
                  gastr_10x_QC = pd.read_csv(Root_Folder  + data_parent_folder + "/Data_ATAC/meta_10X_gastr.csv",',',index_col=[0])
                  gastr_10x_QC = gastr_10x_QC.drop(columns=gastr_10x_QC.columns[gastr_ind_drp],axis=1)

                  gastr_10x_QC_train = gastr_10x_QC.loc[Rna_train.index]
                  gastr_10x_QC_test = gastr_10x_QC.loc[Rna_test.index]

                  gastr_ind_drp = [1,2]
                  gastr_10x_QC1 = pd.read_csv(Root_Folder  + data_parent_folder + "/Data_ATAC/meta_10X_gastr.csv",',',index_col=[0])
                  gastr_10x_QC1 = gastr_10x_QC1.drop(columns=gastr_10x_QC1.columns[gastr_ind_drp],axis=1)

                  gastr_10x_QC1_train = gastr_10x_QC1.loc[Rna_train.index]
                  gastr_10x_QC1_test = gastr_10x_QC1.loc[Rna_test.index]

                  gastr_10x_QC_comb = pd.concat([gastr_10x_QC_train, gastr_10x_QC_test],axis=0)
                  gastr_10x_QC1_comb = pd.concat([gastr_10x_QC1_train, gastr_10x_QC1_test],axis=0)


                  ind_sub = np.asarray(np.random.permutation(52466),dtype=np.int32)
                  gastr_10x_QC_comb = gastr_10x_QC_comb  
                  gastr_10x_QC1_comb = gastr_10x_QC1_comb

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
#Estimates differential directions
lab1 = "E8.5" 
smps_1 = 4483
lab2 = "E8.0" 
smps_2 = 6230
lab3 = "E7.75" 
smps_3 = 17720
lab4 = "E7.5" 
smps_4 = 12876
lab5 = "E7.0" 
smps_5 = 5290
lab6 = "E6.75" 
smps_6 = 2169
lab7 = "E6.5" 
smps_7 = 3697

col_all_1_rna = create_exp(gastr_10x_QC1_comb,lab1,smps_1,lab2,smps_2)
col_all_2_rna = create_exp(gastr_10x_QC1_comb,lab2,smps_2,lab3,smps_3)
col_all_3_rna = create_exp(gastr_10x_QC1_comb,lab3,smps_3,lab4,smps_4)
col_all_4_rna = create_exp(gastr_10x_QC1_comb,lab4,smps_4,lab5,smps_5)
col_all_5_rna = create_exp(gastr_10x_QC1_comb,lab5,smps_5,lab6,smps_6)
col_all_6_rna = create_exp(gastr_10x_QC1_comb,lab6,smps_6,lab7,smps_7)

20.0
40.0
60.0
80.0
100.0
20.0
40.0
60.0
80.0
100.0
20.0
40.0
60.0
80.0
100.0
20.0
40.0
60.0
80.0
100.0
20.0
40.0
60.0
80.0
100.0
20.0
40.0
60.0
80.0
100.0


In [ ]:
cl_1_med_rna,_,_,x1 = collect_stats(col_all_1_rna)
cl_2_med_rna,_,_,x2 = collect_stats(col_all_2_rna)
cl_3_med_rna,_,_,x3 = collect_stats(col_all_3_rna)
cl_4_med_rna,_,_,x4 = collect_stats(col_all_4_rna)
cl_5_med_rna,_,_,x5 = collect_stats(col_all_5_rna)
cl_6_med_rna,_,_,x6 = collect_stats(col_all_6_rna)

In [ ]:
med_1 = pd.DataFrame(np.squeeze(cl_1_med_rna),index=Rna.columns)
med_2 = pd.DataFrame(np.squeeze(cl_2_med_rna),index=Rna.columns)
med_3 = pd.DataFrame(np.squeeze(cl_3_med_rna),index=Rna.columns)
med_4 = pd.DataFrame(np.squeeze(cl_4_med_rna),index=Rna.columns)
med_5 = pd.DataFrame(np.squeeze(cl_5_med_rna),index=Rna.columns)
med_6 = pd.DataFrame(np.squeeze(cl_6_med_rna),index=Rna.columns)

med_1_rng = pd.DataFrame(np.squeeze(x1),index=Rna.columns)
med_1_rng = pd.DataFrame(np.squeeze(x1),index=Rna.columns)
med_2_rng = pd.DataFrame(np.squeeze(x2),index=Rna.columns)
med_3_rng = pd.DataFrame(np.squeeze(x3),index=Rna.columns)
med_4_rng = pd.DataFrame(np.squeeze(x4),index=Rna.columns)
med_5_rng = pd.DataFrame(np.squeeze(x5),index=Rna.columns)
med_6_rng = pd.DataFrame(np.squeeze(x6),index=Rna.columns)

In [ ]:
'''med_1_rng.to_csv(Root_Folder + data_parent_folder + "/scMVI_DDLFC_meta/mESC_10X/med_1_rnd_scMVI_mESC_10X_rna.csv",header=True,sep='\t')
med_2_rng.to_csv(Root_Folder + data_parent_folder + "/scMVI_DDLFC_meta/mESC_10X/med_2_rnd_scMVI_mESC_10X_rna.csv",header=True,sep='\t')
med_3_rng.to_csv(Root_Folder + data_parent_folder + "/scMVI_DDLFC_meta/mESC_10X/med_3_rnd_scMVI_mESC_10X_rna.csv",header=True,sep='\t')
med_4_rng.to_csv(Root_Folder + data_parent_folder + "/scMVI_DDLFC_meta/mESC_10X/med_4_rnd_scMVI_mESC_10X_rna.csv",header=True,sep='\t')
med_5_rng.to_csv(Root_Folder + data_parent_folder + "/scMVI_DDLFC_meta/mESC_10X/med_5_rnd_scMVI_mESC_10X_rna.csv",header=True,sep='\t')
med_6_rng.to_csv(Root_Folder + data_parent_folder + "/scMVI_DDLFC_meta/mESC_10X/med_6_rnd_scMVI_mESC_10X_rna.csv",header=True,sep='\t')

med_1.to_csv(Root_Folder + data_parent_folder + "/scMVI_DDLFC_meta/mESC_10X/Diff_Dir_1_scMVI_mESC_rna.csv",header=True,sep='\t')
med_2.to_csv(Root_Folder + data_parent_folder + "/scMVI_DDLFC_meta/mESC_10X/Diff_Dir_2_scMVI_mESC_rna.csv",header=True,sep='\t')
med_3.to_csv(Root_Folder + data_parent_folder + "/scMVI_DDLFC_meta/mESC_10X/Diff_Dir_3_scMVI_mESC_rna.csv",header=True,sep='\t')
med_4.to_csv(Root_Folder + data_parent_folder + "/scMVI_DDLFC_meta/mESC_10X/Diff_Dir_4_scMVI_mESC_rna.csv",header=True,sep='\t')
med_5.to_csv(Root_Folder + data_parent_folder + "/scMVI_DDLFC_meta/mESC_10X/Diff_Dir_5_scMVI_mESC_rna.csv",header=True,sep='\t')
med_6.to_csv(Root_Folder + data_parent_folder + "/scMVI_DDLFC_meta/mESC_10X/Diff_Dir_6_scMVI_mESC_rna.csv",header=True,sep='\t')'''

'med_1_rng.to_csv(Root_Folder + data_parent_folder + "/scMVI_DDLFC_meta/mESC_10X/med_1_rnd_scMVI_mESC_10X_rna.csv",header=True,sep=\'\t\')\nmed_2_rng.to_csv(Root_Folder + data_parent_folder + "/scMVI_DDLFC_meta/mESC_10X/med_2_rnd_scMVI_mESC_10X_rna.csv",header=True,sep=\'\t\')\nmed_3_rng.to_csv(Root_Folder + data_parent_folder + "/scMVI_DDLFC_meta/mESC_10X/med_3_rnd_scMVI_mESC_10X_rna.csv",header=True,sep=\'\t\')\nmed_4_rng.to_csv(Root_Folder + data_parent_folder + "/scMVI_DDLFC_meta/mESC_10X/med_4_rnd_scMVI_mESC_10X_rna.csv",header=True,sep=\'\t\')\nmed_5_rng.to_csv(Root_Folder + data_parent_folder + "/scMVI_DDLFC_meta/mESC_10X/med_5_rnd_scMVI_mESC_10X_rna.csv",header=True,sep=\'\t\')\nmed_6_rng.to_csv(Root_Folder + data_parent_folder + "/scMVI_DDLFC_meta/mESC_10X/med_6_rnd_scMVI_mESC_10X_rna.csv",header=True,sep=\'\t\')\n\nmed_1.to_csv(Root_Folder + data_parent_folder + "/scMVI_DDLFC_meta/mESC_10X/Diff_Dir_1_scMVI_mESC_rna.csv",header=True,sep=\'\t\')\nmed_2.to_csv(Root_Folder + data